## SIT 112 - Data Science Concepts 


## Tutorial 4

In this tutorial, you will:

* become familiar with numpy package
    * basic representation/transformation of vectors, matrices 
    * compute distances such as Euclidean distance and cosine distance
* construct term-by-document matrix from text data
* apply tf-idf
* retrieve a document
* plot a tagcloud
    * in form of histogram with label
    * in form of word visualization




####Distance between 2 vectors

`Distance` is a numerical description of how far apart objects are. It is a concrete way of describing what it means for elements of some space to be close or far away from each other. For example the distance between two vectors in an n-dimensional space.

Now that you have know how to represent an n-dimensional vector in Python with NumPy arrays, we will write a function as a metric to measure the distance between two vectors. There are multiple ways to measure the distance between two vectors. We will discuss Euclidean distance and cosine distance.

#####Euclidean distance

Euclidean distance comes from Geometry. If we assume $\mathbf{x}_{1}=\left[x_{11},x_{12},\ldots,x_{1n}\right]$ and $\mathbf{x}_{2}=\left[x_{21},x_{22},\ldots,x_{2n}\right]$, then the Euclidean distance between $\mathbf{x}_{1}$ and $\mathbf{x}_{2}$ is defined as:

$$d\left(\mathbf{x}_{1},\mathbf{x}_{2}\right)=\sqrt{\left(x_{11}-x_{21}\right)^{2}+\left(x_{12}-x_{22}\right)^{2}+\ldots+\left(x_{1n}-x_{2n}\right)^{2}}
$$

We can use array operators for this task.

In [ ]:
def euclidean_distance1(x1, x2):
    d = x1 - x2
    d = d ** 2
    return np.sqrt(d.sum())
x1 = np.array([-1, 2, 0, 5])
x2 = np.array([4, 2, 1, 0])
euclidean_distance1(x1, x2)
Since two vectors passed to the function should be the same size, it is better to perform a sanity check before applying the subtraction. Otherwise it will raise an error. We can do this by using `if - elif` statement or as a better practice by using `try - except`.
import sys

def euclidean_distance2(x1, x2):
    if x1.ndim != 1 or x2.ndim != 1:
        sys.exit('either x1 or x2 or both are not vectors')
    elif x1.shape[0] != x2.shape[0]:
        sys.exit('x1 and x2 are not the same size')
    else:
        d = x1 - x2
        d = d ** 2
        return np.sqrt(d.sum())
# fix this cell

x1 = np.array([-1, 2, 0, 5, 9])
x2 = np.array([4, 2, 1, 0])
euclidean_distance2(x1, x2)
def euclidean_distance3(x1, x2):
    try:
        d = x1 - x2
        d = d ** 2
        return np.sqrt(d.sum())
    except ValueError as e:
        print "Vectors passed to the function are not the same size"
        # you can return a default value
        return None
# fix this cell

x1 = np.array([-1, 2, 0, 5, 9])
x2 = np.array([4, 2, 1, 0])
euclidean_distance3(x1, x2)
#####cosine distance and cosine similarity
Cosine similarity is a measure of similarity between two vectors that measures the cosine of the angle between them. Cosine similarity is widely used in information retrieval and text mining as a measure of similarity between documents and is defined as:

$$S_{c}\left(\mathbf{x}_{1},\mathbf{x_{2}}\right)=\frac{\mathbf{x}_{1}.\mathbf{x_{2}}}{\parallel\mathbf{x}_{1}\parallel^{2}+\parallel\mathbf{x}_{2}\parallel^{2}-\mathbf{x}_{1}.\mathbf{x_{2}}}$$


Cosine similarity is particularly used in positive space where the outcome is bounded in [0, 1]. The cosine distance is defined as the complement to cosine similrity in positive space that is $D_{c}\left(x_{1},x_{2}\right)=1-S_{c}\left(x_1,x_2\right)$ where $D_c$ is the cosine distance and $S_c$ is the cosine similarity.
def cosine_distance(x1, x2):
    try:
        num = (x1*x2).sum()
        denom = (x1*x1).sum() + (x2*x2).sum() - (x1*x2).sum()
        num += 0.0    # or use np.astype(float) to make sure of float division
        return 1 - num/denom
    except ValueError as e:
        print "Vectors passed to the function are not the same size"
        return None
x1 = np.array([2, 0, 5, 9])
x2 = np.array([4, 2, 1, 0])
cosine_distance(x1, x2)
np.divide(4, 5)
###Term-by-Document matrix
A term-by-document matrix is a mathematical representation of a text corpus. It describes the frequency of terms that occure in the document collection. Each row corresponds to a document and each column correspond to a term. Thus the value that appears in row $j$ and column $i$ represents the frequency of appearing term $i$ in document $j$. (possibly link it to bag of words?)

We will represent two datasets with term-by-document matrix:

* a collection of 100 Twitter messages about Geelong
* a collection of 6 news articles (5 about Apple and 1 about poletics)

The data is already collected and stores in text files. Thus you will need to:

* read the text files
    * using file opbjetc
* perform pre-processing
    * using string methods
    * using re package
* construct the term-by-document matrix
    * using numpy arrays and operations
####Twitter dataset
First read the data:
import os

# get current working directory
cwd = os.getcwd()   

# join the subdirectory of the data and data file name
file_path = os.path.join(cwd, "data\\prac04\\tweets.txt")

# read the contents of the file and store it in a list
with open(file_path) as fp:
    tweets = fp.readlines()    
for tweet in tweets:
    print tweet
Mostly when dealing with data, we have to perform some sort of data pre-processing. Data collection is often loosely controlled, resulting in out of the range values, missing values, and etc. Thus quality of the data is first and formost before running an analysis. This step is specific to the nature of the data. For example for text data it may consist of cleaning, normalization, tokenization, and etc. 

In this case, our pre-processing consists of:

* converting all the words into lower case to remove the effect of the letter case
* replacing the URLs with a simple string such as 'url'. From the previous cell, you should be able to see that many of the tweets contain a URL. Since we are not using them now, we can remove them or replace them.
* Removing the punctuations
import numpy as np
import re
import string
from collections import Counter
def pre_process(doc):
    """
    pre-processes a doc
      * Converts the tweet into lower case,
      * removes the URLs,
      * removes the punctuations
      * tokenizes the tweet
    """
    
    doc = doc.lower()
    # gettign rid of non ascii codes
    doc = doc.decode('ascii', 'ignore')
    
    # repalcing URLs
    url_pattern = "http://[^\s]+|https://[^\s]+|www.[^\s]+|[^\s]+\.com|bit.ly/[^\s]+"
    doc = re.sub(url_pattern, 'url', doc) 

    punctuation = r"\(|\)|#|\'|\"|-|:|\\|\/|!|_|,|=|;|>|<|\."
    doc = re.sub(punctuation, ' ', doc)
    
    return doc.split()
def termdoc(docs):
    """
    returns the term-by-document matrix and the vocabulary of the passed corpus
    """
    
    vocab = set()   
    termdoc_sparse = []

    for doc in docs:
        # pre-process the doc
        doc_tokens = pre_process(doc)
        # computes the frequencies for doc
        doc_sparse = Counter(doc_tokens)    

        termdoc_sparse.append(doc_sparse)

        # update the vocab
        vocab.update(doc_sparse.iterkeys())  

    vocab = list(vocab)
    vocab.sort()

    n_docs = len(docs)
    n_vocab = len(vocab)
    termdoc_dense = np.zeros((n_docs, n_vocab), dtype=int)

    for j, doc_sparse in enumerate(termdoc_sparse):
        for term, freq in doc_sparse.iteritems():
            termdoc_dense[j, vocab.index(term)] = freq
            
    return termdoc_dense, vocab
tweets_termdoc, tweets_vocab = termdoc(tweets)
Lets look at one of tweets:
j = 0
print tweets[j]
print tweets_termdoc[j]
So baiscally, now each tweet is represented by a vector of size `len(tweets_vocab)`.
####News dataset
Similar to previous sections, the data is stored in text files names as'news1.txt', ..., 'news5.txt'. All we have to do is read the files, construct the corpus and send it to `termdoc()` function.

In [ ]:
n_docs = 6
cwd = os.getcwd()   
news = []

for j in xrange(1, n_docs+1):
    filename = "news{}.txt".format(j)
    file_path = os.path.join(cwd, "data\\prac04\\{}".format(filename))
    with open(file_path) as fp:
        news.append(fp.read())

news_termdoc, news_vocab = termdoc(news)

So now each news article is represented with a large vector of size `len(news_vocab)`. We can do many things with this representation. For example measuring the distance between two documents. The first 5 news articles are tech news and about Apple, but the 6th one is about poletics. We expect that tech news be more similar to each other rather than to the poletics news. In other words the distance between two articles frmom tech news should be less than the distance between a tech news article and a news article about politics. This is shown below:

In [ ]:
print cosine_distance(news_termdoc[1], news_termdoc[2])   # both aout Apple
print cosine_distance(news_termdoc[1], news_termdoc[5])   # one from tech world, the other from politics

###TF-IDF

Term count can reflect the importance of a term in a document. However, it does not consider the effect of document length and how popular the term is across multiple documents in the corpus. That's where **TF-IDF** comes in. It stands for "Term Frequency - Inverse Document Frequency" and it is a way to score the importance of words in document collection. Its concept is very simple and based on two intuitive rules:
 
* if a word appears frequently in a documnt, it is important => Give it a high score
* if a word appears in many documents, it is not a unique identifier => give it a low score

Therefore words such as "the", "for", and "of" which appear in many documents are scored down. They are usually called "stop-words" and since they do not carry any significance, they are removed from the vocabulary. 

There are variations in how we calculate TF. For example raw frequency or log normalization. We will use the frequency which is equal to term count divided by the document length. There are also different ways to calculate IDF. We will use smooth inverse frequency which is defined as
$$\text{IDF}\left(t,D\right)=\log\left(\frac{N_{D}}{1+|d\in D\,:\, t\in d|}\right)$$

Then TF-IDF of a term in a document in a corpus is calculated as TF of the term in the document multipled by the IDF of the word in the corpus.

In [ ]:
def tf_idf(termdoc):
    """
    returns the TF-IDF of a etrm by document matrix
    """
    
    n_docs = termdoc.shape[0]
    n_vocab = termdoc.shape[1]
    
    denom = (termdoc != 0).astype(int)
    denom = 1.0 + denom.sum(axis=0)       
    idf = np.log(n_docs / denom)
    
    row_sums = termdoc.sum(axis=1).astype(float)
    tf = termdoc / row_sums[:, np.newaxis]
    return tf*idf

In [ ]:
tweets_tf_idf = tf_idf(tweets_termdoc)
news_tf_idf = tf_idf(news_termdoc)

In [ ]:
# cosine distance for TF-IDF representation of the corpus

print cosine_distance(news_tf_idf[1], news_tf_idf[2])   # both aout Apple
print cosine_distance(news_tf_idf[1], news_tf_idf[5])   # one from tech world, the other from politics

###Visualization

In this section we write two functions to visualize a document. One using a histogram and the other using a WordCloud. The latter is a visual representation of text data where the importance of each term is shown by its font size.

####Histogram
The package we use for plotting is called `matplotlib`. 

In [ ]:
# make sure figures are shown in ipython notebook
%matplotlib inline    

import matplotlib.pyplot as plt

In [ ]:
def plot_hist(doc, vocab, K=25):
    
    if type(vocab) != np.ndarray:
        vocab = np.array(vocab)
    
    # sorting the indices from max to min
    idx = doc.argsort()[::-1][:K]
    
    x = np.arange(K)
    y = doc[idx]
    labels = vocab[idx]

    fig, ax = plt.subplots(figsize=(12, 7))
    ax.bar(x, y)
    ax.set_xticks(x+0.5)
    ax.set_xticklabels(labels, rotation=90)

    ax.set_xlabel('Terms')
    ax.set_ylabel('TF-IDF weight')
    
    return fig, ax

Now we can plot the histogram of any document in the corpus. For example the first news article. First we plot its count  representation.

In [ ]:
plot_hist(news_termdoc[0], news_vocab, 20)

As you see, since we have not removed the stopwords, the histogram is mostly showing meaningless words. We can not understand what the news is about by looking at the histogram. One remedy is removing stopwords and not using them in the vocabulary. Another remedy is using TF-IDF representation for the visualization.

In [ ]:
plot_hist(news_tf_idf[0], news_vocab, 20)

As the histogram shows, the first news is about a 'streaming' 'service' to listen to 'songs' o 'phones' that 'connects' 'fans' to 'artists'.

####WordCloud

In [ ]:
import wordcloud

In [ ]:
doc = news_tf_idf[0]
vocab = news_vocab
K = 50
width = 500
height = 500
prefer_horiz = 0.9


if type(vocab) != np.ndarray:
    vocab = np.array(vocab)

# sorting the indices from max to min
idx = doc.argsort()[::-1][:K]

freqs = doc[idx]
freqs /= freqs.sum()

word_pairs = [(vocab[idx[i]], freqs[i]) for i in range(K)]
elements = wordcloud.fit_words(word_pairs, width=width, height=height,
                                   prefer_horiz=prefer_horiz)

In [ ]:
wc = wordcloud()